In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.metrics import ConfusionMatrixDisplay, classification_report, RocCurveDisplay
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [2]:
credit = pd.read_csv('../data/train_cleaned.csv')
credit.head()

,ID,is_delinquent,length_of_credit,number_of_delinquent_months,average_delinquency_rate,3mo_delinquency,6mo_delinquency,12mo_delinquency,FLAG_OWN_CAR,FLAG_OWN_REALTY,...,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,AGE,YEARS_EMPLOYED
0,5008804,0,16,2,0.125000,0,0,0,Y,Y,...,Civil marriage,Rented apartment,1,1,0,0,missing,2.0,32,12
1,5008805,0,15,2,0.133333,0,0,0,Y,Y,...,Civil marriage,Rented apartment,1,1,0,0,missing,2.0,32,12
2,5008806,0,30,7,0.233333,0,0,1,Y,Y,...,Married,House / apartment,1,0,0,0,Security staff,2.0,58,3
3,5008808,1,5,2,0.400000,1,1,1,N,Y,...,Single / not married,House / apartment,1,0,1,1,Sales staff,1.0,52,8
4,5008809,0,5,0,0.000000,0,0,0,N,Y,...,Single / not married,House / apartment,1,0,1,1,Sales staff,1.0,52,8


In [3]:
credit.columns

Index(['ID', 'is_delinquent', 'length_of_credit',
       'number_of_delinquent_months', 'average_delinquency_rate',
       '3mo_delinquency', '6mo_delinquency', '12mo_delinquency',
       'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL',
       'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS',
       'NAME_HOUSING_TYPE', 'FLAG_MOBIL', 'FLAG_WORK_PHONE', 'FLAG_PHONE',
       'FLAG_EMAIL', 'OCCUPATION_TYPE', 'CNT_FAM_MEMBERS', 'AGE',
       'YEARS_EMPLOYED'],
      dtype='object')

In [4]:
credit.columns = credit.columns.str.lower()
credit.columns

Index(['id', 'is_delinquent', 'length_of_credit',
       'number_of_delinquent_months', 'average_delinquency_rate',
       '3mo_delinquency', '6mo_delinquency', '12mo_delinquency',
       'flag_own_car', 'flag_own_realty', 'cnt_children', 'amt_income_total',
       'name_income_type', 'name_education_type', 'name_family_status',
       'name_housing_type', 'flag_mobil', 'flag_work_phone', 'flag_phone',
       'flag_email', 'occupation_type', 'cnt_fam_members', 'age',
       'years_employed'],
      dtype='object')

In [5]:
X = credit.drop(columns=[
    'id', 'is_delinquent', 'number_of_delinquent_months', 'average_delinquency_rate', 
    '3mo_delinquency', '6mo_delinquency', '12mo_delinquency'])

y = credit['is_delinquent']

In [6]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
X_train.dtypes

length_of_credit         int64
flag_own_car            object
flag_own_realty         object
cnt_children             int64
amt_income_total       float64
name_income_type        object
name_education_type     object
name_family_status      object
name_housing_type       object
flag_mobil               int64
flag_work_phone          int64
flag_phone               int64
flag_email               int64
occupation_type         object
cnt_fam_members        float64
age                      int64
years_employed           int64
dtype: object

In [8]:
cat_cols = X_train.select_dtypes(include='object').columns
num_cols = X_train.select_dtypes(include=['int64', 'float64']).columns

## Train, test, split and column transformer with 3 different targets

In [9]:
X = credit.drop(columns=[
    'id', 'is_delinquent', 'number_of_delinquent_months', 'average_delinquency_rate', 
    '3mo_delinquency', '6mo_delinquency', '12mo_delinquency'])

y_1 = credit['is_delinquent']
y_2 = credit['3mo_delinquency']
y_3 = credit['6mo_delinquency']

X_train, X_val, y_train_1, y_val_1 = train_test_split(X, y_1, test_size=0.2, random_state=42)
X_train, X_val, y_train_2, y_val_2 = train_test_split(X, y_2, test_size=0.2, random_state=42)
X_train, X_val, y_train_3, y_val_3 = train_test_split(X, y_3, test_size=0.2, random_state=42)

cat_cols = X_train.select_dtypes(include='object').columns
num_cols = X_train.select_dtypes(include=['int64', 'float64']).columns

sc = StandardScaler()

oh = OneHotEncoder(
    categories='auto', 
    drop='first', 
    sparse_output=False, 
    dtype='int',
    handle_unknown = 'ignore'
)

ct = ColumnTransformer(
    transformers=[
        ('oh', oh, cat_cols),
        ('sc', sc, num_cols)
    ]
)

X_train_ct = ct.fit_transform(X_train)
X_val_ct = ct.transform(X_val)

## Gradient Boosting with default parameters
Predicting the classes with 3 different targets

In [48]:
credit['is_delinquent'].value_counts(normalize=True)

0    0.757308
1    0.242692
Name: is_delinquent, dtype: float64

In [27]:
gb = GradientBoostingClassifier()
gb.fit(X_train_ct, y_train_1)

print(gb.score(X_train_ct, y_train_1))
print(gb.score(X_val_ct, y_val_1))

0.778815196394076
0.7633612363168062


In [49]:
credit['3mo_delinquency'].value_counts(normalize=True)

0    0.589633
1    0.410367
Name: 3mo_delinquency, dtype: float64

In [46]:
gb = GradientBoostingClassifier()
gb.fit(X_train_ct, y_train_2)

print(gb.score(X_train_ct, y_train_2))
print(gb.score(X_val_ct, y_val_2))

0.758813586606568
0.7546683837733419


In [50]:
credit['6mo_delinquency'].value_counts(normalize=True)

0    0.526401
1    0.473599
Name: 6mo_delinquency, dtype: float64

In [52]:
gb = GradientBoostingClassifier()
gb.fit(X_train_ct, y_train_3)

print(gb.score(X_train_ct, y_train_3))
print(gb.score(X_val_ct, y_val_3))

0.7706455247907277
0.766580811332904


## Gradient Boosting with GridSearchCV

In [52]:
pgrids = {
    'learning_rate': [.15, .175, 0.2],
    'n_estimators': [285, 290, 295],
    'max_depth': [8, 9, 10]
}

In [53]:
%%time
gb = GradientBoostingClassifier()
gs = GridSearchCV(gb, param_grid=pgrids, cv=5)
gs.fit(X_train_ct, y_train)

CPU times: user 5min 26s, sys: 1.68 s, total: 5min 28s
Wall time: 5min 33s


GridSearchCV(cv=5,
             estimator=GradientBoostingClassifier(learning_rate=0.175,
                                                  max_depth=9),
             param_grid={'n_estimators': [285, 290, 295, 300]})

In [54]:
print(gs.score(X_train_ct, y_train))
print(gs.score(X_val_ct, y_val))

0.9760141661300709
0.7908886027044431


In [102]:
gs.best_params_

{'n_estimators': 290}

## Ada Boost with GridSearchCV

In [ ]:
%%time
tree = DecisionTreeClassifier(random_state=123)#, max_depth=1
ada = AdaBoostClassifier(estimator=tree, random_state=42)

pgrids_ada = {
    'learning_rate': [2.25, 2.5, 2.7],
    'n_estimators': [250, 300, 350],
    'estimator__max_depth': [71, 81, 91, 101, None],
    # 'estimator__min_samples_leaf': [7, 8, 9],
    'estimator__max_features': ['auto', 'sqrt', 'log2']
}


gs_ada = GridSearchCV(ada, param_grid=pgrids_ada, cv=5)
gs_ada.fit(X_train_ct, y_train)

print(gs_ada.score(X_train_ct, y_train))
print(gs_ada.score(X_val_ct, y_val))

In [ ]:
gs_ada.best_params_

In [58]:
%%time
tree = DecisionTreeClassifier(random_state=123)#, max_depth=1
ada = AdaBoostClassifier(estimator=tree, random_state=42)

pgrids_ada = {
    'learning_rate': [2.25, 2.5, 2.7],
    'n_estimators': [250, 300, 350],
    'estimator__max_depth': [71, 81, 91, 101, None],
    # 'estimator__min_samples_leaf': [7, 8, 9],
    'estimator__max_features': ['auto', 'sqrt', 'log2']
}


gs_ada_2 = GridSearchCV(ada, param_grid=pgrids_ada, cv=5)
gs_ada_2.fit(X_train_ct, y_train_2)

print(gs_ada.score(X_train_ct, y_train_2))
print(gs_ada.score(X_val_ct, y_val_2))

KeyboardInterrupt: 


## SCV

In [41]:
credit['is_delinquent'].value_counts(normalize=True)

0    0.757308
1    0.242692
Name: is_delinquent, dtype: float64

In [42]:
svc_1 = SVC(C=1, kernel='rbf', degree = 1)
svc_1.fit(X_train_ct, y_train_1)
# Evaluate model.
svc_1.score(X_val_ct, y_val_1)

0.7548293625241468

In [43]:
credit['3mo_delinquency'].value_counts(normalize=True)

0    0.589633
1    0.410367
Name: 3mo_delinquency, dtype: float64

In [44]:
svc_2 = SVC(C=1, kernel='rbf', degree = 1)
svc_2.fit(X_train_ct, y_train_2)
# Evaluate model.
svc_2.score(X_val_ct, y_val_2)

0.7598197037990985

In [53]:
credit['6mo_delinquency'].value_counts(normalize=True)

0    0.526401
1    0.473599
Name: 6mo_delinquency, dtype: float64

In [54]:
svc_3 = SVC(C=1, kernel='rbf', degree = 1)
svc_3.fit(X_train_ct, y_train_3)
# Evaluate model.
svc_3.score(X_val_ct, y_val_3)

0.7670637475853187

## Logistic Regression

In [11]:
print(credit['is_delinquent'].value_counts(normalize=True))
print(credit['3mo_delinquency'].value_counts(normalize=True))
print(credit['6mo_delinquency'].value_counts(normalize=True))

0    0.757308
1    0.242692
Name: is_delinquent, dtype: float64
0    0.589633
1    0.410367
Name: 3mo_delinquency, dtype: float64
0    0.526401
1    0.473599
Name: 6mo_delinquency, dtype: float64


In [32]:
param_grid = {
    'C':  np.linspace(1, 10,5),
    'penalty': ['l2'],
    'solver': ['lbfgs'],
    'max_iter': [500,1000]
    
}

gs = GridSearchCV(
estimator= LogisticRegression(),
param_grid=param_grid)


gs.fit(X_train_ct, y_train_1)
print(f'Training score on is_delinquent : {gs.score(X_train_ct, y_train_1)}')
print(f'Testing score on is_delinquent : {gs.score(X_val_ct, y_val_1)}')
gs_1_best_params = gs.best_params_
print("Best parameters for is_delinquent:", gs_1_best_params)

gs.fit(X_train_ct, y_train_2)
print(f'Training score on 3mo_delinquency : {gs.score(X_train_ct, y_train_2)}')
print(f'Testing score on 3mo_delinquency : {gs.score(X_val_ct, y_val_2)}')
gs_2_best_params = gs.best_params_
print("Best parameters for 3mo_delinquency:", gs_2_best_params)

gs.fit(X_train_ct, y_train_3)
print(f'Training score on 6mo_delinquency : {gs.score(X_train_ct, y_train_3)}')
print(f'Testing score on 6mo_delinquency : {gs.score(X_val_ct, y_val_3)}')
gs_3_best_params = gs.best_params_
print("Best parameters for 6mo_delinquency:", gs_3_best_params)


Training score on is_delinquent : 0.7627575660012879
Testing score on is_delinquent : 0.754507405022537
Best parameters for is_delinquent: {'C': 10.0, 'max_iter': 500, 'penalty': 'l2', 'solver': 'lbfgs'}
Training score on 3mo_delinquency : 0.749637797810689
Testing score on 3mo_delinquency : 0.7435608499678042
Best parameters for 3mo_delinquency: {'C': 10.0, 'max_iter': 500, 'penalty': 'l2', 'solver': 'lbfgs'}
Training score on 6mo_delinquency : 0.7563184159690921
Testing score on 6mo_delinquency : 0.7549903412749517
Best parameters for 6mo_delinquency: {'C': 1.0, 'max_iter': 500, 'penalty': 'l2', 'solver': 'lbfgs'}


## Random Forest

In [35]:
param_grid = {
    'n_estimators': [300,400],
    'max_depth': [None],
    'min_samples_split': [5,8],
    'min_samples_leaf': [3, 4],
    'max_features': ['sqrt']
}

rf = RandomForestClassifier()

gs = GridSearchCV(estimator=rf, param_grid=param_grid)

gs.fit(X_train_ct, y_train_1)
print(f'Training score on is_delmin_samples_leafinquent : {gs.score(X_train_ct, y_train_1)}')
print(f'Testing score on is_delinquent : {gs.score(X_val_ct, y_val_1)}')
gs_1_best_params = gs.best_params_
print("Best parameters for is_delinquent:", gs_1_best_params)

gs.fit(X_train_ct, y_train_2)
print(f'Training score on 3mo_delinquency : {gs.score(X_train_ct, y_train_2)}')
print(f'Testing score on 3mo_delinquency : {gs.score(X_val_ct, y_val_2)}')
gs_2_best_params = gs.best_params_
print("Best parameters for 3mo_delinquency:", gs_2_best_params)

gs.fit(X_train_ct, y_train_3)
print(f'Training score on 6mo_delinquency : {gs.score(X_train_ct, y_train_3)}')
print(f'Testing score on 6mo_delinquency : {gs.score(X_val_ct, y_val_3)}')
gs_3_best_params = gs.best_params_
print("Best parameters for 6mo_delinquency:", gs_3_best_params)

Training score on is_delmin_samples_leafinquent : 0.8959272376046362
Testing score on is_delinquent : 0.793947198969736
Best parameters for is_delinquent: {'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 200}
Training score on 3mo_delinquency : 0.8967723760463618
Testing score on 3mo_delinquency : 0.7974887314874437
Best parameters for 3mo_delinquency: {'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 300}
Training score on 6mo_delinquency : 0.89210399227302
Testing score on 6mo_delinquency : 0.7981326464906632
Best parameters for 6mo_delinquency: {'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 300}
